<a href="https://colab.research.google.com/github/deepshikharbhardwaj/Machine_Learning/blob/main/Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Change Path

In [2]:
!pwd

/content


In [3]:
%cd '/content/drive/MyDrive/PyTorch/Custom_Dataset'

/content/drive/MyDrive/PyTorch/Custom_Dataset


#Imports

In [15]:
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from skimage import io
from torch.utils.data import (Dataset,DataLoader,)  # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets  # Standard datasets
import torch.nn.functional as F

#CNN

In [5]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels,out_channels=8,kernel_size=(3, 3),stride=(1, 1),padding=(1, 1),)
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3, 3),stride=(1, 1),padding=(1, 1),)
        self.fc1 = nn.Linear(16 * 8 * 8, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

#Set Device

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Hyperparameter

In [7]:
learning_rate = 1e-4
batch_size = 64
num_epochs = 5

#Load Model

In [8]:
model = CNN(in_channels=3, num_classes=10)
model.classifier = nn.Sequential(nn.Linear(512, 100), nn.ReLU(), nn.Linear(100, 10))
model.to(device)

CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1024, out_features=10, bias=True)
  (classifier): Sequential(
    (0): Linear(in_features=512, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)

#Augmentation

In [9]:
my_transforms = transforms.Compose([  
        transforms.Resize((36, 36)),            # Resizes (32,32) to (36,36)
        transforms.RandomCrop((32, 32)),        # Takes a random (32,32) crop
        transforms.ColorJitter(brightness=0.5), # Change brightness of image
        transforms.RandomRotation(degrees=45),  # Perhaps a random rotation from -45 to 45 degrees
        transforms.RandomHorizontalFlip(p=0.5), # Flips the image horizontally with probability 0.5
        transforms.RandomVerticalFlip(p=0.05),  # Flips image vertically with probability 0.05
        transforms.RandomGrayscale(p=0.2),      # Converts to grayscale with probability 0.2
        transforms.ToTensor(),                  # Finally converts PIL image to tensor so we can train w. pytorch
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Note: these values aren't optimal
    ])

#Load Dataset

In [18]:
train_dataset = datasets.CIFAR10(root="content/", train=True, transform=my_transforms, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.CIFAR10(root="content/", train=False, transform=my_transforms, download=True)
test_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


#Loss and Optimizer

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#Train Network

In [16]:
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

Cost at epoch 0 is 2.1515441230495873
Cost at epoch 1 is 1.9843560737722061
Cost at epoch 2 is 1.9235509108094608
Cost at epoch 3 is 1.885553022479767
Cost at epoch 4 is 1.8567193469123158


#Check Accuracy

In [19]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

Checking accuracy on Training Set
Got 17186 / 50000 with accuracy 34.37
Checking accuracy on Test Set
Got 17251 / 50000 with accuracy 34.50
